In [42]:
import nfl_data_py as nfl
import pandas as pd

In [4]:
years = range(1999, 2023 + 1)

In [5]:
df = nfl.import_pbp_data(years)

1999 done.
2000 done.
2001 done.
2002 done.
2003 done.
2004 done.
2005 done.
2006 done.
2007 done.
2008 done.
2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
Downcasting floats.


In [6]:
df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,offense_players,defense_players,n_offense,n_defense,ngs_air_yards,time_to_throw,was_pressure,route,defense_man_zone_type,defense_coverage_type
0,35.0,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60.0,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,82.0,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,103.0,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,126.0,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Cells of interest:
- third_down_converted
- fg_prob
- td_prob
- 'vegas_wpa'
- 'vegas_home_wpa'
- 'home_wp_post'
- 'vegas_wp'
- 'vegas_home_wp'
- 'receiver_player_id'
- 'receiver_player_name'
- drive_inside20
- spread_line
- total_line

In [98]:
df_redzone = df.query("yardline_100 <= 20 and play_type == 'pass' and season_type == 'REG'")[['season', 
                                                                                               'receiver_player_name', 'receiver_id']]

In [99]:
df_redzone_targets = df_redzone \
    .groupby(by=['season', 'receiver_id']) \
    .agg(redzone_targets=('receiver_player_name', "count"))

df_redzone_targets.reset_index(inplace=True)
df_redzone_targets.rename(columns={"receiver_id": "player_id"}, inplace=True)

In [100]:
df_redzone_targets.head()

,season,player_id,redzone_targets
0,1999,00-0000003,3
1,1999,00-0000145,1
2,1999,00-0000166,7
3,1999,00-0000172,1
4,1999,00-0000210,2


In [101]:
df_core = pd.read_pickle("../../FantasyData/data-frames/df_basic_ngs_snaps_adv_1999_2023.pkl")

In [102]:
df_core.head()

,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,yds_pfr_rush,td_pfr_rush,x1d_pfr_rush,ybc_pfr_rush,ybc_att_pfr_rush,yac_pfr_rush,yac_att_pfr_rush,brk_tkl_pfr_rush,att_br_pfr_rush,loaded_pfr_rush
0,00-0000003,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00-0000003,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00-0000003,2000,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00-0000007,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00-0000007,2000,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
df_merge = df_core.merge(df_redzone_targets, on=["season", "player_id"], how="left")

In [110]:
df_merge.query("redzone_targets >= 30")[["season", "player_name", "redzone_targets", "targets"]].head()

,season,player_name,redzone_targets,targets
989,2000,Marvin Harrison,31.0,168
1662,2002,Randy Moss,36.0,184
1677,2002,Eric Moulds,31.0,180
5089,2011,Roddy White,30.0,180
5747,2009,Brandon Marshall,30.0,154


In [112]:
df_merge['redzone_targets'].fillna(0, inplace=True)

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_11075/2923396243.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merge['redzone_targets'].fillna(0, inplace=True)


# Exploring

In [34]:
df['play_type'].iloc[39]

'no_play'

In [35]:
df['desc'].iloc[39]

'(2:27) D.Pederson pass incomplete to C.Johnson. PENALTY on ARI-Z.Walz, Roughing the Passer, 3 yards, enforced at ARI 6 - No Play.'

In [33]:
df[['yardline_100', 'td_prob', 'no_score_prob', 'drive_inside20', 'desc', 'receiver', 'play_type']].head(50)

,yardline_100,td_prob,no_score_prob,drive_inside20,desc,receiver,play_type
0,30.0,0.334211,0.009824,0.0,C.Jacke kicks 68 yards from ARI 30 to PHI 2. A...,None,kickoff
1,77.0,0.342473,0.009388,0.0,(15:00) D.Pederson pass incomplete to D.Staley...,D.Staley,pass
2,77.0,0.310568,0.009247,0.0,(15:00) D.Staley up the middle to PHI 24 for 1...,None,run
3,76.0,0.262984,0.009103,0.0,"(15:00) PENALTY on PHI-K.Turner, False Start, ...",None,no_play
4,81.0,0.224716,0.009585,0.0,(15:00) D.Pederson pass to B.Finneran to PHI 2...,B.Finneran,pass
5,71.0,0.239127,0.010293,0.0,"(15:00) S.Landeta punts 55 yards to ARI 16, Ce...",None,punt
6,81.0,0.279041,0.015006,0.0,(13:07) A.Murrell right end to ARI 20 for 1 ya...,None,run
7,80.0,0.254127,0.014607,0.0,(13:07) A.Murrell up the middle to ARI 24 for ...,None,run
8,76.0,0.234744,0.014975,0.0,(13:07) J.Plummer pass incomplete to F.Sanders.,F.Sanders,pass
9,76.0,0.161151,0.015543,0.0,"(13:07) S.Player punts 35 yards to PHI 41, Cen...",None,punt


In [10]:
df.columns[:70]

Index(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       'side_of_field', 'yardline_100', 'game_date',
       'quarter_seconds_remaining', 'half_seconds_remaining',
       'game_seconds_remaining', 'game_half', 'quarter_end', 'drive', 'sp',
       'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo', 'ydsnet',
       'desc', 'play_type', 'yards_gained', 'shotgun', 'no_huddle',
       'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 'pass_length',
       'pass_location', 'air_yards', 'yards_after_catch', 'run_location',
       'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result',
       'two_point_conv_result', 'home_timeouts_remaining',
       'away_timeouts_remaining', 'timeout', 'timeout_team', 'td_team',
       'td_player_name', 'td_player_id', 'posteam_timeouts_remaining',
       'defteam_timeouts_remaining', 'total_home_score', 'total_away_score',
       'posteam_score'

In [12]:
df.columns[69:120]

Index(['td_prob', 'extra_point_prob', 'two_point_conversion_prob', 'ep', 'epa',
       'total_home_epa', 'total_away_epa', 'total_home_rush_epa',
       'total_away_rush_epa', 'total_home_pass_epa', 'total_away_pass_epa',
       'air_epa', 'yac_epa', 'comp_air_epa', 'comp_yac_epa',
       'total_home_comp_air_epa', 'total_away_comp_air_epa',
       'total_home_comp_yac_epa', 'total_away_comp_yac_epa',
       'total_home_raw_air_epa', 'total_away_raw_air_epa',
       'total_home_raw_yac_epa', 'total_away_raw_yac_epa', 'wp', 'def_wp',
       'home_wp', 'away_wp', 'wpa', 'vegas_wpa', 'vegas_home_wpa',
       'home_wp_post', 'away_wp_post', 'vegas_wp', 'vegas_home_wp',
       'total_home_rush_wpa', 'total_away_rush_wpa', 'total_home_pass_wpa',
       'total_away_pass_wpa', 'air_wpa', 'yac_wpa', 'comp_air_wpa',
       'comp_yac_wpa', 'total_home_comp_air_wpa', 'total_away_comp_air_wpa',
       'total_home_comp_yac_wpa', 'total_away_comp_yac_wpa',
       'total_home_raw_air_wpa', 'total_away

In [15]:
df.columns[119:179]

Index(['punt_blocked', 'first_down_rush', 'first_down_pass',
       'first_down_penalty', 'third_down_converted', 'third_down_failed',
       'fourth_down_converted', 'fourth_down_failed', 'incomplete_pass',
       'touchback', 'interception', 'punt_inside_twenty', 'punt_in_endzone',
       'punt_out_of_bounds', 'punt_downed', 'punt_fair_catch',
       'kickoff_inside_twenty', 'kickoff_in_endzone', 'kickoff_out_of_bounds',
       'kickoff_downed', 'kickoff_fair_catch', 'fumble_forced',
       'fumble_not_forced', 'fumble_out_of_bounds', 'solo_tackle', 'safety',
       'penalty', 'tackled_for_loss', 'fumble_lost', 'own_kickoff_recovery',
       'own_kickoff_recovery_td', 'qb_hit', 'rush_attempt', 'pass_attempt',
       'sack', 'touchdown', 'pass_touchdown', 'rush_touchdown',
       'return_touchdown', 'extra_point_attempt', 'two_point_attempt',
       'field_goal_attempt', 'kickoff_attempt', 'punt_attempt', 'fumble',
       'complete_pass', 'assist_tackle', 'lateral_reception', 'lateral

In [16]:
df.columns[178:238]

Index(['rushing_yards', 'lateral_receiver_player_id',
       'lateral_receiver_player_name', 'lateral_receiving_yards',
       'lateral_rusher_player_id', 'lateral_rusher_player_name',
       'lateral_rushing_yards', 'lateral_sack_player_id',
       'lateral_sack_player_name', 'interception_player_id',
       'interception_player_name', 'lateral_interception_player_id',
       'lateral_interception_player_name', 'punt_returner_player_id',
       'punt_returner_player_name', 'lateral_punt_returner_player_id',
       'lateral_punt_returner_player_name', 'kickoff_returner_player_name',
       'kickoff_returner_player_id', 'lateral_kickoff_returner_player_id',
       'lateral_kickoff_returner_player_name', 'punter_player_id',
       'punter_player_name', 'kicker_player_name', 'kicker_player_id',
       'own_kickoff_recovery_player_id', 'own_kickoff_recovery_player_name',
       'blocked_player_id', 'blocked_player_name',
       'tackle_for_loss_1_player_id', 'tackle_for_loss_1_player_name'

In [19]:
df.columns[237:290]

Index(['assist_tackle_4_player_name', 'assist_tackle_4_team',
       'tackle_with_assist', 'tackle_with_assist_1_player_id',
       'tackle_with_assist_1_player_name', 'tackle_with_assist_1_team',
       'tackle_with_assist_2_player_id', 'tackle_with_assist_2_player_name',
       'tackle_with_assist_2_team', 'pass_defense_1_player_id',
       'pass_defense_1_player_name', 'pass_defense_2_player_id',
       'pass_defense_2_player_name', 'fumbled_1_team', 'fumbled_1_player_id',
       'fumbled_1_player_name', 'fumbled_2_player_id', 'fumbled_2_player_name',
       'fumbled_2_team', 'fumble_recovery_1_team', 'fumble_recovery_1_yards',
       'fumble_recovery_1_player_id', 'fumble_recovery_1_player_name',
       'fumble_recovery_2_team', 'fumble_recovery_2_yards',
       'fumble_recovery_2_player_id', 'fumble_recovery_2_player_name',
       'sack_player_id', 'sack_player_name', 'half_sack_1_player_id',
       'half_sack_1_player_name', 'half_sack_2_player_id',
       'half_sack_2_player_nam

In [20]:
df.columns[289:340]

Index(['series_success', 'series_result', 'order_sequence', 'start_time',
       'time_of_day', 'stadium', 'weather', 'nfl_api_id', 'play_clock',
       'play_deleted', 'play_type_nfl', 'special_teams_play', 'st_play_type',
       'end_clock_time', 'end_yard_line', 'fixed_drive', 'fixed_drive_result',
       'drive_real_start_time', 'drive_play_count', 'drive_time_of_possession',
       'drive_first_downs', 'drive_inside20', 'drive_ended_with_score',
       'drive_quarter_start', 'drive_quarter_end', 'drive_yards_penalized',
       'drive_start_transition', 'drive_end_transition',
       'drive_game_clock_start', 'drive_game_clock_end',
       'drive_start_yard_line', 'drive_end_yard_line', 'drive_play_id_started',
       'drive_play_id_ended', 'away_score', 'home_score', 'location', 'result',
       'total', 'spread_line', 'total_line', 'div_game', 'roof', 'surface',
       'temp', 'wind', 'home_coach', 'away_coach', 'stadium_id',
       'game_stadium', 'aborted_play'],
      dtype='o

In [21]:
df.columns[339:395]

Index(['aborted_play', 'success', 'passer', 'passer_jersey_number', 'rusher',
       'rusher_jersey_number', 'receiver', 'receiver_jersey_number', 'pass',
       'rush', 'first_down', 'special', 'play', 'passer_id', 'rusher_id',
       'receiver_id', 'name', 'jersey_number', 'id', 'fantasy_player_name',
       'fantasy_player_id', 'fantasy', 'fantasy_id', 'out_of_bounds',
       'home_opening_kickoff', 'qb_epa', 'xyac_epa', 'xyac_mean_yardage',
       'xyac_median_yardage', 'xyac_success', 'xyac_fd', 'xpass', 'pass_oe',
       'nflverse_game_id', 'possession_team', 'offense_formation',
       'offense_personnel', 'defenders_in_box', 'defense_personnel',
       'number_of_pass_rushers', 'players_on_play', 'offense_players',
       'defense_players', 'n_offense', 'n_defense', 'ngs_air_yards',
       'time_to_throw', 'was_pressure', 'route', 'defense_man_zone_type',
       'defense_coverage_type'],
      dtype='object')